# Gemma Quantization 방법


## 양자화 방법 1.
- 허깅페이스에서 다운로드 후 양자화 -> 허깅페이스 재업로드

In [ ]:
!pip instlal huggingface_hub

In [ ]:
import huggingface_hub

huggingface_hub.login()

In [ ]:
!pip install --upgrade trasnformers bitsandbytes accelerate

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed, BitstAndBytesConfig

set_seed(1234)

prompt = "The best recipe for pasta is"

#checkpoint = "googl/gemma-7b"
checkpoint = "wonik-hi/gemma-2b-it-ko_v2"

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4biy_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, quantization_config=bnb_config, device_map="cuda")

inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, do_sample=True, max_new_tokens=150)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(result)

In [ ]:
# 양자화
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "wonik-hi/gemma-2b-it-ko_v2"
quant_path = "gemma-2b-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name, quantization_config=bnb_config, trust_remote_code=True
)

model.save_pretrained("./"+quant_path, safetensors=True)
tokenizer.save_pretrained("./"+quant_path)

model.push_to_hub("wonik-hi/gemma-2b-bnb-4bit")
tokenizer.push_to_hub("wonik-hi/gemma-2b-bnb-4bit")

In [ ]:
# Harness로 평가
!pip install git+https://github.com/EleutherAI/lm-evaluation-harness.git
!pip install auto-gptq optimum autoawq

In [ ]:
!lm_eval --model hf --model_args pretrained=wonik-hi/gemma-2b-bnb-4bit --tasks winogrande,hellaswag,arc_challenge --device cuda:0 --num_fewshot 1 --batch_size 8 --output_path ./eval_harness/gemma-2b-bnb-4bit

In [ ]:
!lm_eval --model hf --model_args pretrained=wonik-hi/gemma-2b-bnb-4bit --tasks winogrande,hellaswag,arc_challenge --device cuda:0 --num_fewshot 5 --batch_size 8 --output_path ./eval_harness/gemma-2b-5shot